In [1]:
!pip install keras
import pandas as pd
from collections import Counter
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# import torch
# import torch.nn as nn
import matplotlib.pyplot as plt 

Using TensorFlow backend.


In [2]:
df=pd.read_csv('Label10000.csv')
print(len(df))
# df = df[df['text'].str.split().str.len().gt(5)]
# df = df[df['text'].str.split().str.len().lt(200)]
df=df.reset_index(drop=None)
print(len(df))
# news=df['text']
# label=df['label']
# len(news)
# current_words=[]
# for i in range(len(news)):
#   # for word in news.loc[i].split(" "):
#     current_words.append(news.loc[i])
# print((current_words[0]))

10000
10000


In [3]:
df=df[['text','label']]
df.loc[1]
# df=df[20000::]
# df=df.reset_index(drop=None)
# df.head()
# df=df[['text','label']]
df.head()


,text,label
0,nape summit also featur annual chariti luncheo...,1
1,appl huge anticip iphon x samsung comeback tou...,0
2,iphon x best phone money isnt issu appl one ex...,0
3,appl aapl premium devic start cool per month d...,0
4,xs spoton face id facialrecognit technolog fan...,0


In [4]:
df['word_count']=df['text'].apply(lambda x: len(str(x).split(" ")))
df.word_count.describe()
common_words=pd.Series(''.join(df['text']).split()).value_counts()
print(common_words[:10])
print("Not Common words")
print(common_words[-10000:])

appl      5971
aapl      2096
inc       2077
share     1790
nasdaq    1219
buy        978
iphon      973
rate       963
report     949
price      917
dtype: int64
Not Common words
ibb           3
narrow        3
andv          3
fold          3
nktr          3
             ..
yearmiss      1
eanotabl      1
naley         1
banksay       1
rumrosappl    1
Length: 10000, dtype: int64


In [5]:
from nltk.tokenize import word_tokenize
most_common=common_words[:3]
least_common=common_words[-1000:]
texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if not (word in most_common or word in least_common):
            final_words.append(word)
    return final_words
for i in range(len(df)):
    if i%10000==0:
        print("------>",i)
    words=word_tokenize(df.loc[i].text)
    # print(df.loc[i].text)
    words=removeWords(words)
    extracted_sentence=' '.join(word for word in words)
    # print(df.loc[i].text)
    texts.append(extracted_sentence)
    labels.append(df.loc[i]['label'])
new_df=pd.DataFrame({
    'text':texts,
    'label':labels
})


------> 0


In [6]:
new_df.loc[4]

text     xs spoton face id facialrecognit technolog fan...
label                                                    0
Name: 4, dtype: object

In [7]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
def pad(x, length=None):
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')

In [8]:
tokenize_text=[]
news_label=[]
for n_z in new_df['text']:
  tokenize_text.append(n_z)
for l in new_df['label']:
  news_label.append(l)

print(tokenize_text[1])
print(news_label[0])

huge anticip iphon x samsung comeback follow note debacl big year smartphon industri
1


In [9]:
text_tokenized, text_tokenizer=tokenize(tokenize_text)
text_padded=pad(text_tokenized)

In [10]:
print(text_tokenized[0])

[2512, 2009, 37, 366, 592, 1706, 2513, 473, 2514, 2515, 135, 19, 514, 1200, 2516, 200, 938, 2517, 579, 229, 824, 2518, 2519, 2520, 1841, 760, 2219, 474, 110, 117, 1707, 2010, 40, 22, 413, 2220, 2521, 352, 2522, 1842, 761]


In [11]:
print((text_padded.shape))
print((text_padded[0]))
trainX=text_padded
print(trainX.shape)
trainY = pd.get_dummies(news_label).values
print(trainY[0])

(10000, 1275)
[2512 2009   37 ...    0    0    0]
(10000, 1275)
[0 1]


In [12]:
# from numpy import zeros
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# t = Tokenizer()
# t.fit_on_texts(current_words)
# vocab_size = len(t.word_index) + 1
# encoded_docs = t.texts_to_sequences(current_words)
# max_length = trainX[0].shape[0]
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [15]:
from sklearn.model_selection import train_test_split
# trainY = pd.get_dummies(label.values).values
# print(trainY.shape)
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=56)

In [16]:
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
testX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
trainX.shape

(8000, 1, 1275)

In [19]:
# !pip install tensorflow==1.14.0
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.optimizers import Adam
from keras.layers import Embedding
from keras.optimizers import SGD
opt=SGD(lr=0.001)
model = Sequential()
# e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1500, trainable=False)
# model.add(e)
model.add(Bidirectional(LSTM(128,input_shape=(1,X_train.shape[1]),dropout=0.2)))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model.fit(trainX, y_train, epochs=40, batch_size=64, verbose=2,validation_split=0.2)

Train on 6400 samples, validate on 1600 samples
Epoch 1/40
 - 1s - loss: 0.6901 - accuracy: 0.5417 - val_loss: 0.6657 - val_accuracy: 0.6050
Epoch 2/40
 - 1s - loss: 0.6508 - accuracy: 0.6556 - val_loss: 0.6454 - val_accuracy: 0.6612
Epoch 3/40
 - 1s - loss: 0.6327 - accuracy: 0.6831 - val_loss: 0.6380 - val_accuracy: 0.6681
Epoch 4/40
 - 1s - loss: 0.6241 - accuracy: 0.6928 - val_loss: 0.6343 - val_accuracy: 0.6794
Epoch 5/40
 - 1s - loss: 0.6196 - accuracy: 0.6925 - val_loss: 0.6324 - val_accuracy: 0.6800
Epoch 6/40
 - 1s - loss: 0.6203 - accuracy: 0.6947 - val_loss: 0.6324 - val_accuracy: 0.6806
Epoch 7/40
 - 1s - loss: 0.6133 - accuracy: 0.7016 - val_loss: 0.6316 - val_accuracy: 0.6831
Epoch 8/40
 - 1s - loss: 0.6142 - accuracy: 0.7008 - val_loss: 0.6323 - val_accuracy: 0.6844
Epoch 9/40
 - 1s - loss: 0.6139 - accuracy: 0.7006 - val_loss: 0.6333 - val_accuracy: 0.6844
Epoch 10/40
 - 1s - loss: 0.6115 - accuracy: 0.7022 - val_loss: 0.6331 - val_accuracy: 0.6844
Epoch 11/40
 - 1s - l

In [20]:
score, acc= model.evaluate(testX, y_test,verbose=0)
print('Test score:', score)
print('Test accuracy:', acc)
# print('F1 score:', f1_score)
# print('Precision:', precision)
# print('Recall:', recall)

Test score: 0.6164590644836426
Test accuracy: 0.7014999985694885


In [21]:
from sklearn.metrics import classification_report
y_pred = model.predict(testX)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred_b = np.argmax(y_test, axis=1)
print(classification_report(y_pred_b, y_pred_bool))

precision    recall  f1-score   support

           0       0.40      0.01      0.01       595
           1       0.70      1.00      0.82      1405

    accuracy                           0.70      2000
   macro avg       0.55      0.50      0.42      2000
weighted avg       0.61      0.70      0.58      2000

